In [1]:
import json
import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
import keras.preprocessing.text as kpt
import keras
import re 
import numpy, time
import random
import pickle
from keras.models import load_model, model_from_json
import random

height has been deprecated.



Using TensorFlow backend.


In [2]:
def transform_data_for_tokenizer(recipe_item):
    original = recipe_item.lower()
    original = re.sub(r'\s*(\d+|[./+*-])', '', original)
    original_split = original.split(" ")
    remove_list = list(set(original).symmetric_difference(units_of_food_recipe))

    keep_list = []
    for i in original_split:
        if i not in food_size and i not in units_of_food_recipe:
            keep_list.append(i)
    
    original_split = list(filter(None, keep_list))
    new = " ".join(original_split)
    return new

In [3]:
units_of_food_recipe = ["cups", "cup", "c.", "c", 
                        "fl. oz.", "fl oz", "fluid ounce", "fluid ounces",
                        "gal", "gal.", "gallon", "gallons",
                        "oz", "oz.", "ounce", "ounces", "ouncs",
                        "pt", "pt.", "pint", "pints",
                        "lb", "lb.", "pound", "pounds",
                        "qt", "qt.", "qts", "qts.", "quart", "quarts",
                        "tbsp.", "tbsp", "T", "T.", "tablespoon", "tablespoons", "tbs.", "tbs",
                        "tsp.", "tsp", "t", "t.", "teaspoon", "teaspoons",
                        "g", "g.", "gr", "gr.", "gram", "grams", "gramme", "grammes",
                        "kg", "kg.", "kilogram", "kilograms", "kilogramme", "kilogrammes",
                        "l", "l.", "liter", "liters", "litre", "litres",
                        "mg", "mg.", "milligram", "milligrams", "milligramme", "milligrammes",
                        "ml", "ml.", "milliliter", "milliliters", "millilitre", "millilitres",
                        "pinch", "pinches", "dash", "dashes", "touch", "touches", "handful", "handfuls",
                        "stick", "sticks",
                        "cans", "can",
                        "to taste",
                        "scoop", "scoops",
                        "dollop", "dollops",
                        "sprig", "sprigs",
                       "recipe",
                       "garnish", "garnished",
                       "sprinkle", "spinkled",
                       "slices",
                       "serving", "servings",
                       "ribs", "rib", "stalk",
                       "inch", "inches", "in.",
                       "drizzle", "drizzled",
                       "to taste"]

food_size = ["small", "medium",
              "large", "about"]

food_format = ["slice", "sliced",
              "dice", "diced",
              "peel", "peeled",
              "chop", "chopped",
              "julienne", "julienned",
              "trim", "trimmed",
              "sift", "sifted",
              "freshly",
              "fine", "finely",
              "drain", "drained",
              "super thinly", "thin", "thinly",
              "grate", "grated",
              "seeded",
              "rinse", "rinsed",
              "smash", "smashed",
              "membrane", "removed",
              "cleaned",
              "mince", "minced",
              "crush", "crushed",
              "about", "around",
              '<hr>']


def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [4]:
nlp_df = pd.read_csv("../../data/nutrient/compiled/train_nn_df.csv", index_col=0)
master_df = pd.read_csv("../../data/nutrient/compiled/nutrition_master_df.csv")

with open('../../data/recipe/recipe_all.json') as f:
    recipe_all = json.load(f)
    
with open('/root/w210_capstone/data/models/NDB_tag_unique_unique_dict.json') as f:
        NDB_tag_unique_unique_dict = json.load(f)

# loading
with open('/root/w210_capstone/data/models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    
# load json and create model
json_file = open('/root/w210_capstone/data/models/model_simple_nn.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("/root/w210_capstone/data/models/model_simple_nn_WEIGHTS.h5")
print("Loaded model from disk")

ambiguous = pd.read_csv("/root/w210_capstone/data/recipe/ambiguous_conversion.csv")

Loaded model from disk


In [9]:























random.seed(time.time())
ids = np.unique(list(recipe_all.keys()))
random.shuffle(ids)

original_text_list = []
NDB_ID_list = []
Description_list = []

for recipe_id in ids[:5]:
    print('\n')
    print(recipe_id)
    
    for ingredient in recipe_all[recipe_id]['ingredients']:
        print(ingredient)
        ingredient_filtered = transform_data_for_tokenizer(ingredient)
        tokens = tokenizer.texts_to_matrix([ingredient_filtered])
        n = 3
        prediction = loaded_model.predict(np.array(tokens))

        index_top_3_list =  loaded_model.predict(np.array(tokens)).argsort()[0][-n:][::-1]
        NDB_top_3_list = []
        percentage_top_3_list = []
        for i in index_top_3_list:   
            for key in NDB_tag_unique_unique_dict.keys():        
                if NDB_tag_unique_unique_dict[key] == i:
                    NDB_top_3_list.append(key)
                    percentage_top_3_list.append(prediction[0][i])

        itr = 0
        while itr < 3:            
            print('\t', NDB_top_3_list[itr], round(percentage_top_3_list[itr], 4), master_df[master_df['NDB_NO'] == "\"{}\"".format(NDB_top_3_list[itr].strip('"'))]['Description'].get_values()[0])
            itr += 1
        
        print('\n')
        
        raw_input_return = input()
        
        if raw_input_return == 'exit':
            break
        
        else:
            if raw_input_return:
                if raw_input_return != 'np.nan':
                    original_text_list.append(ingredient)
                    NDB_ID_list.append(raw_input_return)
                    Description_list.append(master_df[master_df['NDB_NO'] == "\"{}\"".format(raw_input_return.strip('"'))]['Description'].get_values()[0])
            else:
                if raw_input_return != 'np.nan':
                    original_text_list.append(ingredient)
                    NDB_ID_list.append(NDB_top_3_list[0])
                    Description_list.append(master_df[master_df['NDB_NO'] == "\"{}\"".format(NDB_top_3_list[0].strip('"'))]['Description'].get_values()[0])
                

temp_df = pd.DataFrame([original_text_list, NDB_ID_list, Description_list]).transpose()
temp_df.columns = nlp_df.columns
temp_df = temp_df.dropna()
nlp_df_new = pd.concat([nlp_df, temp_df])
nlp_df_new.to_csv("../../data/nutrient/compiled/train_nn_df.csv")
nlp_df = pd.read_csv("../../data/nutrient/compiled/train_nn_df.csv", index_col=0)



RECIPE_79711
1 dried Chipotle chile
	 "11819" 0.8452 peppers, hot chili, red, raw
	 "02031" 0.0931 spices, pepper, red or cayenne
	 "11820" 0.02 peppers, hot chili, red, canned, excluding seeds, solids and liquids



3/4 cup coarsely ground yellow cornmeal
	 "20020" 0.988 cornmeal, whole-grain, yellow
	 "02006" 0.0022 spices, cardamom
	 "02013" 0.002 spices, coriander seed



1/4 cup all purpose flour
	 "20581" 0.9976 wheat flour, white, all-purpose, enriched, unbleached
	 "20082" 0.0013 wheat flour, white, all-purpose, self-rising, enriched
	 "20080" 0.0004 wheat flour, whole-grain (includes foods for usda's food distribution program)



1 teaspoon double-acting baking powder
	 "18370" 0.8694 leavening agents, baking powder, double-acting, straight phosphate
	 "18371" 0.0621 leavening agents, baking powder, low-sodium
	 "11157" 0.0219 chrysanthemum, garland, raw



1 1/2 tablespoons sugar
	 "19335" 0.9831 sugars, granulated
	 "28293" 0.0088 cookie, butter or sugar, with chocolate ic


1 shallot fine dice
	 "11677" 0.9984 shallots, raw
	 "01033" 0.0003 cheese, parmesan, hard
	 "02020" 0.0002 spices, garlic powder



1 sprig rosemary leaves chopped fine
	 "02063" 0.9871 rosemary, fresh
	 "02036" 0.0095 spices, rosemary, dried
	 "18973" 0.0011 focaccia, italian flatbread, plain



10 ounce shiitake mushrooms stems removed + caps sliced
	 "11238" 0.9648 mushrooms, shiitake, raw
	 "11260" 0.0133 mushrooms, white, raw
	 "06625" 0.0029 smart soup, vietnamese carrot lemongrass



1 splash of sherry vinegar
	 "14096" 0.7839 alcoholic beverage, wine, table, red
	 "09152" 0.0464 lemon juice, raw
	 "14536" 0.029 alcoholic beverage, wine, dessert, dry


"02068"
1 tsp tomato paste
	 "11529" 0.5632 tomatoes, red, ripe, raw, year round average
	 "11547" 0.3541 tomato products, canned, puree, without salt added
	 "11546" 0.0245 tomato products, canned, paste, without salt added (includes foods for usda's food distribution program)


"11546"
2 tsp worcestershire sauce
	 "06971" 0.07

In [6]:
original_text_list

['1 1/2 cups crushed graham crackers (about 10 crackers)',
 '7 tablespoons unsalted butter melted',
 '2 tablespoons light brown sugar',
 '2 cups whole milk',
 '1 cup heavy cream',
 '1/2 cup sugar',
 '1/4 cup unsweetened cocoa powder',
 'Pinch salt',
 '1/4 cup cornstarch',
 '4 ounces bittersweet chocolate finely chopped',
 '1 1/2 teaspoons vanilla extract',
 '10 large marshmallows halved on the diagonal',
 '3 ounces shallots chopped',
 '2 tablespoons fresh parsley',
 '1 tablespoon capers',
 'Juice from 1/2 lemon',
 '4 large eggs',
 '2 tablespoons crab base',
 '2 teaspoons seafood seasoning such as Old Bay',
 '1 tablespoon Worcestershire sauce',
 '1 teaspoon ground mustard',
 '1 teaspoon salt',
 '1 teaspoon ground white pepper',
 '2 cups mayonnaise',
 '2 pounds jumbo lump crabmeat',
 '2 pounds super lump crabmeat',
 '2 cups panko breadcrumbs',
 'Oil for greasing the baking sheet',
 '1 tablespoon oil blend',
 '1/2 cup sliced carrots',
 '2 teaspoons minced fresh garlic',
 '2 teaspoons minc